In [ ]:
# pip install git+https://github.com/facebookresearch/HolisticTraceAnalysis.git -v
# doc: https://pytorch.ac.cn/tutorials/beginner/hta_intro_tutorial.html
from hta.trace_analysis import TraceAnalysis
analyzer = TraceAnalysis(trace_dir = "../profiler_log/")

In [ ]:
"""
doc: https://hta.readthedocs.io/en/latest/source/features/temporal_breakdown.html
为了有效利用 GPU，了解它们在特定作业上花费时间的方式至关重要。它们主要用于计算、通信、内存事件，还是处于空闲状态？时间分解功能详细分析了在这三个类别中花费的时间。

空闲时间 - GPU 处于空闲状态。
计算时间 - GPU 用于矩阵乘法或向量运算。
非计算时间 - GPU 用于通信或内存事件。

为了实现高训练效率，代码应最大化计算时间，最小化空闲时间和非计算时间。以下函数生成一个 dataframe，详细列出了每个 rank 的时间使用分解。
"""
time_spent_df = analyzer.get_temporal_breakdown()
print(time_spent_df)

In [ ]:
"""
doc: https://hta.readthedocs.io/en/latest/source/features/idle_time_breakdown.html
了解 GPU 空闲的时间量及其原因有助于指导优化策略。当 GPU 上没有运行内核时，它被认为是空闲的。我们开发了一种算法，将空闲时间分为三个不同的类别

Host 等待： 指的是 GPU 上的空闲时间，这是由于 CPU 入队内核不够快，无法使 GPU 得到充分利用。可以通过检查导致速度变慢的 CPU 操作符、增加批处理大小和应用操作符融合来解决这些低效问题。
Kernel 等待： 这指的是在 GPU 上启动连续内核时相关的短暂开销。可以通过使用 CUDA Graph 优化来最小化归因于此类别的空闲时间。
其他等待： 此类别包括当前无法归因的空闲时间，原因可能是信息不足。可能的原因包括使用 CUDA 事件在 CUDA 流之间同步以及启动内核时的延迟。

Host 等待时间可以解释为 GPU 由于 CPU 停顿的时间。要将空闲时间归因于 Kernel 等待，我们使用以下启发式方法

连续内核之间的间隔 < 阈值

默认阈值值为 30 纳秒，可以使用 consecutive_kernel_delay 参数进行配置。默认情况下，空闲时间分解仅计算 rank 0。为了计算其他 rank 的分解，请使用 get_idle_time_breakdown 函数中的 ranks 参数。空闲时间分解可以按如下方式生成
"""
idle_time_df = analyzer.get_idle_time_breakdown()
print(idle_time_df)

In [ ]:
"""
doc: https://hta.readthedocs.io/en/latest/source/features/kernel_breakdown.html
Kernel 分解功能分解了每种内核类型（例如通信 (COMM)、计算 (COMP) 和内存 (MEM)）
在所有 rank 上花费的时间，并显示了在每个类别中花费的时间比例。这是每个类别所花费时间的百分比饼图
"""
kernel_type_metrics_df, kernel_metrics_df = analyzer.get_gpu_kernel_breakdown()
print(kernel_type_metrics_df, kernel_metrics_df)

In [ ]:
"""
doc: https://hta.readthedocs.io/en/latest/source/features/comm_comp_overlap.html#
在分布式训练中，大量时间花费在 GPU 之间的通信和同步事件上。
为了实现高 GPU 效率（例如 TFLOPS/GPU），关键在于通过计算内核保持 GPU 超负荷运转。
换句话说，GPU 不应因未解决的数据依赖而阻塞。衡量计算被数据依赖阻塞程度的一种方法是计算通信计算重叠。
如果通信事件与计算事件重叠，则观察到更高的 GPU 效率。通信和计算重叠不足会导致 GPU 空闲，从而导致效率低下。
总而言之，期望有更高的通信计算重叠。
为了计算每个 rank 的重叠百分比，我们测量以下比率

（通信时花费在计算上的时间）/（花费在通信上的时间）

通信计算重叠可以按如下方式计算
"""
overlap_df = analyzer.get_comm_comp_overlap()
print(overlap_df)

In [ ]:
"""
doc: https://hta.readthedocs.io/en/latest/source/features/augmented_counters.html
内存带宽计数器测量通过内存复制 (memcpy) 和内存设置 (memset) 事件在 H2D、D2H 和 D2D 之间复制数据时使用的内存复制带宽。
HTA 还计算每个 CUDA 流上的未完成操作数。
我们将此称为队列长度。
当流上的队列长度达到 1024 或更大时，新事件无法在该流上调度，CPU 将停顿，直到 GPU 流上的事件处理完毕。

generate_trace_with_counters API 输出一个带有内存带宽和队列长度计数器的新跟踪文件。
新跟踪文件包含指示 memcpy/memset 操作使用的内存带宽的跟踪以及每个流上队列长度的跟踪。
默认情况下，这些计数器是使用 rank 0 的跟踪文件生成的，新文件的名称中包含后缀 _with_counters。
用户可以选择使用 generate_trace_with_counters API 中的 ranks 参数为多个 rank 生成计数器。
"""
analyzer.generate_trace_with_counters()

# generate summary
mem_bw_summary = analyzer.get_memory_bw_summary()
queue_len_summary = analyzer.get_queue_length_summary()

# get time series
mem_bw_series = analyzer.get_memory_bw_time_series()
queue_len_series = analyzer.get_queue_length_time_series()

print(mem_bw_summary,queue_len_summary,mem_bw_series,queue_len_series)

In [ ]:
"""
doc: https://hta.readthedocs.io/en/latest/source/features/cuda_kernel_launch_stats.html
短 GPU 内核 - 持续时间小于 相应的 CPU 运行时事件。
运行时事件异常值 - 持续时间过长的 CPU 运行时事件。
Launch delay outliers （启动延迟异常值） - 计划时间过长的 GPU 内核。
"""
kernel_info_df = analyzer.get_cuda_kernel_launch_stats()
print(kernel_info_df)

In [ ]:
"""
doc: https://hta.readthedocs.io/en/latest/source/features/frequent_cuda_kernels.html
此功能可查找为任何 指定的运算符。
它会生成一个新的跟踪文件，该文件覆盖了前 k 个 在原始跟踪文件上识别的模式。
在新跟踪文件中搜索关键字会突出显示相关的 CPU 和 GPU作。
这 突出显示的事件表明在哪里寻找融合 CUDA 的机会 内核或 CPU作。
"""
cuda_sequences_df = analyzer.get_frequent_cuda_kernel_sequences(
    operator_name = "aten::linear",
    output_dir = "/tmp/"
)

In [ ]:
"""
doc: https://hta.readthedocs.io/en/latest/source/features/trace_diff.html
"""


In [ ]:
"""
doc: https://hta.readthedocs.io/en/latest/source/features/cupti_counter_analysis.html
"""
gpu_kernels = analyzer.get_cupti_counter_data_with_operators(ranks=[0])[0]


In [ ]:
"""
doc: hhttps://hta.readthedocs.io/en/latest/source/features/lightweight_critical_path_analysis.html
"""
